<a href="https://colab.research.google.com/github/Minsoo1036/Algorithm-Study/blob/main/NeuralProphet_prac1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if "google.colab" in str(get_ipython()):
    # uninstall preinstalled packages from Colab to avoid conflicts
    !pip uninstall -y torch notebook notebook_shim tensorflow tensorflow-datasets prophet torchaudio torchdata torchtext torchvision
    !pip install git+https://github.com/ourownstory/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes

import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

set_log_level("ERROR")

Found existing installation: torch 2.3.0
Uninstalling torch-2.3.0:
  Successfully uninstalled torch-2.3.0
  Cloning https://github.com/ourownstory/neural_prophet.git to /tmp/pip-req-build-samg26ay
  Running command git clone --filter=blob:none --quiet https://github.com/ourownstory/neural_prophet.git /tmp/pip-req-build-samg26ay
  Resolved https://github.com/ourownstory/neural_prophet.git to commit 23543560b4ed278e84d1fd0f119d332342336d0d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.14 requires torchvision>=0.11, which is not installed.
fastai 2.7.14 requires torch<2.3,>=1.10, but you have torch 2.3.0 which is incompatible.


ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.
ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/rawdata/jecheol_coal_counterfactual_df_update.csv")

In [ ]:
df.head()

,Unnamed: 0,Date,Data,d1,d2
0,0,2006-01-01,NaN,4057.0,100.000
1,1,2006-02-01,NaN,3834.0,100.791
2,2,2006-03-01,NaN,4219.0,100.757
3,3,2006-04-01,NaN,4049.0,99.412
4,4,2006-05-01,NaN,4104.0,95.557


In [ ]:
df.tail()

,Unnamed: 0,Date,Data,d1,d2
200,200,2022-09-01,265.477,9900.0,121.600
201,201,2022-10-01,290.226,10500.0,123.526
202,202,2022-11-01,282.148,10400.0,119.901
203,203,2022-12-01,262.821,10600.0,116.699
204,204,2023-01-01,316.276,NaN,114.917


In [ ]:
df = df.tail(157)
df = df.head(156)
df

,Unnamed: 0,Date,Data,d1,d2
48,48,2010-01-01,192.500,5691.0,88.193
49,49,2010-02-01,224.000,5213.0,90.285
50,50,2010-03-01,232.500,5845.0,89.946
51,51,2010-04-01,237.500,5831.0,90.014
52,52,2010-05-01,247.500,5860.0,93.832
...,...,...,...,...,...
199,199,2022-08-01,243.636,10200.0,117.591
200,200,2022-09-01,265.477,9900.0,121.600
201,201,2022-10-01,290.226,10500.0,123.526
202,202,2022-11-01,282.148,10400.0,119.901


In [ ]:
df = df[['Date','Data']]
train_df=df.rename(columns={'Date':'ds','Data':'y'})
train_df

,ds,y
48,2010-01-01,192.500
49,2010-02-01,224.000
50,2010-03-01,232.500
51,2010-04-01,237.500
52,2010-05-01,247.500
...,...,...
199,2022-08-01,243.636
200,2022-09-01,265.477
201,2022-10-01,290.226
202,2022-11-01,282.148


In [ ]:
train_df = train_df.head(156-26-26+13) #train + valid data (13개 일부러 더 줌.)
train_df

,ds,y
48,2010-01-01,192.500
49,2010-02-01,224.000
50,2010-03-01,232.500
51,2010-04-01,237.500
52,2010-05-01,247.500
...,...,...
160,2019-05-01,207.358
161,2019-06-01,196.632
162,2019-07-01,181.964
163,2019-08-01,155.324


In [ ]:
train_df = train_df.reset_index()
train_df = train_df.drop("index",axis=1)
train_df

,ds,y
0,2010-01-01,192.500
1,2010-02-01,224.000
2,2010-03-01,232.500
3,2010-04-01,237.500
4,2010-05-01,247.500
...,...,...
112,2019-05-01,207.358
113,2019-06-01,196.632
114,2019-07-01,181.964
115,2019-08-01,155.324


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import random
score_record = {}
n_trial = 5

for z in range(n_trial):

    print(str(z+1) + " / " +  str(n_trial))
    random.seed(z+1)
    opt_dict ={'input_len': random.choice([12,24,36,48]),
                'growth': random.choice(['linear','off']), 'n_changepoints': random.choice([2,4,6,8,10]),
                'changepoints_range': random.choice([0.5,0.6,0.7,0.8,0.9]), 'trend_reg': random.choice([0,0.05,0.1]),
               'ar_reg': random.choice([0.0001,0.001,0.01,0.1,0.5]),
               'ar_num_layers' : random.choice([1,2]),
               'ar_hidden_dim' : random.choice([2,4,8,16,32]),
                'learning_rate': random.choice([0.0001,0.001,0.01])}

    print(opt_dict)

    import torch
    torch.manual_seed(0)

    input_len = opt_dict['input_len']
    m = NeuralProphet(
        #trend
        growth=opt_dict['growth'],
        n_changepoints = opt_dict['n_changepoints'],
        changepoints_range = opt_dict['changepoints_range'],
        trend_reg = opt_dict['trend_reg'],
        trend_reg_threshold = False,
        #seasonality
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode = 'additive',
        seasonality_reg = 0,
        #AR
        n_lags=input_len,
        ar_reg = opt_dict['ar_reg'],
        #deep
        ar_layers = np.array([opt_dict['ar_hidden_dim']] * opt_dict['ar_num_layers']),
        lagged_reg_layers = np.array([opt_dict['ar_hidden_dim']] * opt_dict['ar_num_layers']),
        #model architecture, optimizer
        n_forecasts = 26,
        learning_rate = opt_dict['learning_rate'],
        epochs = None,
        batch_size = None,
        loss_func = 'Huber',
        optimizer = 'AdamW',
        quantiles = [0.05*q for q in range(1,20,1)]
    )
    df_train = train_df.iloc[:91,]
    df_test = train_df.iloc[91-input_len:]

    metrics = m.fit(df_train, freq="M", progress="bar")
    forecast = m.predict(df_test)

    y_valid_pred = []
    for k in range(13):
        y_valid_pred.append(forecast.iloc[input_len+k,2+k])
    #metrics = m.fit(df_train, validation_df=df_test, freq="M", progress="bar")

    y_valid_pred = np.array(y_valid_pred)
    y_valid_real = np.array(forecast.iloc[input_len:input_len+13,1])

    print("valid mape : ", np.mean(abs((y_valid_pred - y_valid_real) / y_valid_real)))

1 / 5
{'input_len': 24, 'growth': 'linear', 'n_changepoints': 6, 'changepoints_range': 0.5, 'trend_reg': 0.05, 'ar_reg': 0.1, 'ar_num_layers': 2, 'ar_hidden_dim': 16, 'learning_rate': 0.0001}


Training: 0it [00:00, ?it/s]

Predicting: 6it [00:00, ?it/s]

valid mape :  0.45562037913783904
2 / 5
{'input_len': 12, 'growth': 'linear', 'n_changepoints': 2, 'changepoints_range': 0.7, 'trend_reg': 0, 'ar_reg': 0.01, 'ar_num_layers': 2, 'ar_hidden_dim': 32, 'learning_rate': 0.0001}


Training: 0it [00:00, ?it/s]

...

Predicting: 6it [00:00, ?it/s]

valid mape :  0.501526909244191
5 / 5
{'input_len': 36, 'growth': 'off', 'n_changepoints': 10, 'changepoints_range': 0.5, 'trend_reg': 0.05, 'ar_reg': 0.001, 'ar_num_layers': 1, 'ar_hidden_dim': 4, 'learning_rate': 0.0001}


Training: 0it [00:00, ?it/s]

Predicting: 4it [00:00, ?it/s]

valid mape :  0.4448348411674736
